# Rust-Python Binding Notebook Course

In [2]:
%pip install -r requirements.txt
!maturin develop --release


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
🔗 Found pyo3 bindings
🐍 Found CPython 3.10 at /home/peter/miniconda3/bin/python
📡 Using build options features from pyproject.toml
   Compiling pyo3-build-config v0.19.2
   Compiling pyo3-ffi v0.19.2========> ] 422/431: pyo3-build-config           
   Compiling pyo3 v0.19.2
   Compiling arrow v46.0.0========> ] 428/431: pyo3                           
   Compiling blogpost_ffi v0.1.0 (/home/peter/Documents/work/blogpost_ffi)    
    Finished release [optimized] target(s) in 3.46slogpost_ffi                
📦 Built wheel for CPython 3.10 to /tmp/.tmp3eYPVU/blogpost_ffi-0.1.0-cp310-cp310-linux_x86_64.whl
🛠 Installed blogpost_ffi-0.1.0


In [3]:
import blogpost_ffi
import os
import pyarrow as pa
import time
import tracing
from opentelemetry.trace.propagation.tracecontext import (
    TraceContextTextMapPropagator,
)

In [4]:
pa.array([])
value = [1] * 100_000_000

In [ ]:
# Default implementation

print(f"---Default Implementation---")

start_time = time.time()

array = blogpost_ffi.create_list(value)

print(f"default: {time.time() - start_time:.3f}s")

In [ ]:
# PyBytes Implementation

print(f"---PyBytes Implementation---")

value_bytes = bytes(value)

start_time = time.time()

array = blogpost_ffi.create_list_bytes(value_bytes)

print(f"bytes: {time.time() - start_time:.3f}s")


In [ ]:
# Arrow Implementation

print(f"---Arrow Implementation---")

value_arrow = pa.array(value, type=pa.uint8())

start_time = time.time()

array = blogpost_ffi.create_list_arrow(value_arrow)

print(f"arrow: {time.time() - start_time:.3f}s")

In [ ]:
# Debugging eyre

print(f"---Eyre error---")

# Try:
# array = blogpost_ffi.create_list_arrow(1)
#
## This error panics the whole program and is therefore uncatchable.
ERROR_WITHOUT_EYRE = """
thread '<unnamed>' panicked at 'called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError('Expected instance of pyarrow.lib.Array, got builtins.int'), traceback: None }', src/lib.rs:45:62
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
Traceback (most recent call last):
  File "/home/peter/Documents/work/blogpost_ffi/test_script.py", line 79, in <module>
    array = blogpost_ffi.create_list_arrow(1)
pyo3_runtime.PanicException: called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError('Expected instance of pyarrow.lib.Array, got builtins.int'), traceback: None }
"""


# Eyre result
array = blogpost_ffi.create_list_arrow_eyre(1)

---Eyre error---


NameError: name 'blogpost_ffi' is not defined

In [ ]:
# Eyre default

def abc():
    assert False, "I have no idea what is wrong"

In [ ]:
print(f"---Eyre no traceback---")

array = blogpost_ffi.call_func_eyre(abc)

In [ ]:
# Eyre traceback
print(f"---Eyre traceback---")

array = blogpost_ffi.call_func_eyre_traceback(abc)

In [ ]:
# Unbounded Memory Growth
print(f"-->Open a Memory analyzer")
array = blogpost_ffi.unbounded_memory_growth()

In [ ]:

# Unbounded Memory Growth
print(f"-->Open a Memory analyzer")
array = blogpost_ffi.bounded_memory_growth()

In [ ]:
# GIL Lock
print(f"---GIL Lock---")

array = blogpost_ffi.gil_lock()

In [ ]:
# GIL unlock
print(f"---GIL Lock---")

array = blogpost_ffi.gil_unlock()

In [ ]:
!docker run -d -p6831:6831/udp -p6832:6832/udp -p16686:16686 jaegertracing/all-in-one:latest

ddf9aa05cd69251df39873189b15b25b4f29fe6173c063b2aa1b3478653c3f93
docker: Error response from daemon: driver failed programming external connectivity on endpoint stupefied_wu (5d49720e7638c2c2f107cff084c214c9d1f6ddffab801695bfbfa95f091aa8df): Bind for 0.0.0.0:16686 failed: port is already allocated.


In [ ]:
# Global tracing
print(f"---Global tracing---")

def abc(cx):
    propagator = TraceContextTextMapPropagator()
    context = propagator.extract(carrier=cx)

    with tracing.tracer.start_as_current_span(
        name="Python_span", context=context
    ) as child_span:
        child_span.add_event("in Python!")
        output = {}
        tracing.propagator.inject(output)
        time.sleep(2)
    return output


array = blogpost_ffi.global_tracing(abc)

In [1]:
# Distributing the code

!maturin publish --non-interactive

error: unexpected argument '--dry-run' found

  note: to pass '--dry-run' as a value, use '-- --dry-run'

Usage: maturin publish [OPTIONS] [ARGS]...

For more information, try '--help'.


In [4]:
!maturin  generate-ci github --zig --platform all                                                       

🔗 Found pyo3 bindings
# This file is autogenerated by maturin v1.2.3
# To update, run
#
#    maturin generate-ci github --zig --platform all
#
name: CI

on:
  push:
    branches:
      - main
      - master
    tags:
      - '*'
  pull_request:
  workflow_dispatch:

permissions:
  contents: read

jobs:
  linux:
    runs-on: ubuntu-latest
    strategy:
      matrix:
        target: [x86_64, x86, aarch64, armv7, s390x, ppc64le]
    steps:
      - uses: actions/checkout@v3
      - uses: actions/setup-python@v4
        with:
          python-version: '3.10'
      - name: Build wheels
        uses: PyO3/maturin-action@v1
        with:
          target: ${{ matrix.target }}
          args: --release --out dist --find-interpreter --zig
          sccache: 'true'
          manylinux: auto
      - name: Upload wheels
        uses: actions/upload-artifact@v3
        with:
          name: wheels
          path: dist

  windows:
    runs-on: windows-latest
    strategy:
      matrix:
        target